In [1]:
#In this notebook i took idbm review dataset and try to build LSTM model

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [3]:
data.shape

(50000, 2)

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [7]:
data.isnull().sum()

,0
review,0
sentiment,0


In [8]:
data["review"][5]

'Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.'

In [9]:
import re

In [10]:
##Preprocessing Step
#1 remove html taggs
#2 remove number
#2 remove punctuations
#3 lower text
#4 remove stopword

#6 remove singular char

In [11]:
contractions = {
    "can't": "cannot",
    "won't": "will not",
    "i'm": "i am",
    "you're": "you are",
    "it's": "it is",
    "they're": "they are",
    "let's": "let us",
    "don't": "do not",
    "isn't": "is not",
    "i've": "i have",
    "i'll": "i will",
    "didn't": "did not",
    "you've": "you have",
    "i'd": "i would",
    "he's": "he is",
    "she's": "she is",
    # Add more contractions as needed
}

In [12]:
from gensim.parsing.preprocessing import remove_stopwords

In [13]:
html_pattern = re.compile(r"<[^>]+>")

alpha_pattern = re.compile(r"[^\D]")

contractions_pattern = re.compile(r'\b(' + '|'.join(contractions.keys()) + r')\b')

In [14]:
from gensim.utils import simple_preprocess

In [15]:
def preprocessing (text): # if contaction mapping needed add contrations in argument which define above
  sentence = text.lower()
  sentence = html_pattern.sub("",sentence)
  sentence = alpha_pattern.sub("",sentence)
  sentence = remove_stopwords(sentence)
  # sentence= contractions_pattern.sub(lambda x: contractions[x.group()], text) ## If contraction mapping needed

  sentence = simple_preprocess(sentence)
  sentence = ' '.join([word for word in sentence if len(word) > 2])

  return sentence


In [16]:
df = data.copy()

In [17]:
df["review"] = df["review"].apply(preprocessing)

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
tokenizer = Tokenizer(num_words=800,oov_token='<OOV>',split=' ')
tokenizer.fit_on_texts(df["review"].values)


X = tokenizer.texts_to_sequences(df["review"].values)

X = pad_sequences(X)

In [20]:
X.shape

(50000, 1303)

In [21]:
# y = df["sentiment"]

In [22]:
y = np.array(np.where(df["sentiment"]=="positive",1,0))

In [23]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train ,X_test , y_train, y_test = train_test_split(
                                                    X,y,
                                                    test_size=0.25,
                                                    random_state=42)

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding ,Dense,LSTM

In [39]:
# 500 = Vocab length
# 120 = We can give this any thing for neuron

In [40]:
model = Sequential()

model.add(Embedding(800,120,input_length =1303))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))

In [41]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [42]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [44]:
model.fit(X_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.6635 - loss: 0.6050
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 37s 53ms/step - accuracy: 0.8407 - loss: 0.3616
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.8564 - loss: 0.3384
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.8570 - loss: 0.3398
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.8709 - loss: 0.3059
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.8729 - loss: 0.2990
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.8765 - loss: 0.2907
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.8793 - loss: 0.2826
Epoch 9/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 0.8854 - loss: 0.2743
Epoch 10/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.8845 - loss: 0.2700


In [46]:
y_test

array([1, 1, 0, ..., 1, 1, 1])

In [62]:
y_pred = model.predict(X_test)

391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step


In [64]:
y_pred =y_pred.flatten()

In [65]:
y_pred = np.where(y_pred>0.5,1,0)

In [66]:
y_pred

array([0, 1, 0, ..., 0, 1, 1])

In [67]:
from sklearn.metrics import accuracy_score

In [68]:
accureacy = accuracy_score(y_test,y_pred,)

In [69]:
accureacy

0.8484

In [1]:
m = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=data.target_names,
            yticklabels=data.target_names)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

NameError: name 'confusion_matrix' is not defined